In [1]:
import pickle
import networkx

In [17]:
with open('base_data/G_california.p', 'rb') as f:
    G = pickle.load(f)

In [43]:
print("Number of nodes : ", G.number_of_nodes())
print("Number of edges : ", G.number_of_edges())
print("\n10 first nodes :\n", "\t"+ "\n\t".join([str(node) for node in list(G.nodes())[:10]]))
print("\n10 first edges :\n", "\t" +"\n\t".join([str(edge) for edge in list(G.edges())[:10]]))

Number of nodes :  9816
Number of edges :  16424

10 first nodes :
 	hortau
	spiritual-gangster
	brit
	seatninja
	bluedata-software
	village-defense
	netherfire-entertainment
	dubuc-motors
	reflektive
	bundled-bliss

10 first edges :
 	('hortau', 'inv_advantage-capital-partners')
	('hortau', 'inv_avrio-capital')
	('spiritual-gangster', 'inv_m3-ventures')
	('brit', 'inv_shervin-pishevar')
	('brit', 'inv_general-catalyst-partners')
	('brit', 'inv_jim-felding')
	('brit', 'inv_cowboy-ventures')
	('brit', 'inv_marissa-mayer')
	('brit', 'inv_jennifer-hyman')
	('brit', 'inv_ff-angel-llc')


In [57]:
investors = [node for node in list(G.nodes()) if node[:4]=='inv_']
ventures = [node for node in list(G.nodes()) if node[:4]!='inv_']
print("Extracted {0} ventures and {1} investors so {2} (/{3}) nodes ".format(len(investors), len(ventures), len(investors)+len(ventures),  G.number_of_nodes()))

Extracted 5883 ventures and 3933 investors so 9816 (/9816) nodes 


In [53]:
def 

['hortau',
 'spiritual-gangster',
 'brit',
 'seatninja',
 'bluedata-software',
 'village-defense',
 'netherfire-entertainment',
 'dubuc-motors',
 'reflektive',
 'bundled-bliss',
 'ross-intelligence',
 'second-measure',
 'visiquate',
 'sentio-solutions-inc',
 'barry-s-bootcamp',
 'minio-inc',
 'angellist',
 'placeavote-com',
 'august',
 'setpoint-medical',
 'hommily',
 'transfer-devices',
 'whipclip',
 'sentrian',
 'gtxcel',
 'synedgen',
 'fanbread',
 'rex-real-estate-exchange',
 'precision-credit-group',
 'zycada-networks',
 'earcrush',
 'hashplay',
 'acalvio-technologies',
 'bitmovin',
 'gitlab-com',
 'skymind',
 'levyx',
 'wagespot',
 'liquidity-nanotech',
 'guardian-birthcare',
 'robinhealth',
 'weblife-balance',
 'ever-2',
 'tvpage-inc',
 'testlio',
 'phantom-login-3',
 'suggestic',
 'alpha-moda-lab',
 'epinomics',
 'pocketgm',
 'doordash',
 'primus-power',
 'tzoa',
 'trumed-systems',
 'zugata',
 'norse-corporation',
 'ring',
 'piper-2',
 'nimbleheart',
 'stance',
 'crowdstrike',
 